### Imports

In [99]:
import numpy as np
import pandas as pd 
from collections import Counter
import os
import glob
import copy
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
from sklearn import metrics
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering

### Read every cohort study file

In [100]:
datasets = [pd.read_csv(file, index_col=0, low_memory=False) for file in sorted(glob.glob('../cohort_studies_full_data/' + "/*."+'csv'))]
cohorts = [file.split(".")[0] for file in sorted(os.listdir('../cohort_studies_full_data/'))]

In [101]:
# make a dictionary that contains all cohorts as a dataframe
cohort_studies = dict()
# dfsss = dict()

for cohort, dataset in zip(cohorts, datasets):
    cohort_n = cohort.split("_MERGE")[0]
    cohort_studies[cohort_n] = dataset.loc[dataset['Months']==0].copy() # reduce to BL visit
#     dfsss[cohort_n] = dataset

In [102]:
datasets_sub = [pd.read_csv(file, index_col=0, low_memory=False) for file in sorted(glob.glob('../preprocessed_datasets/' + "/*."+'csv'))]
cohorts_sub = [file.split(".")[0] for file in sorted(os.listdir('../preprocessed_datasets/'))]

In [103]:
# make a dictionary that contains all cohorts as a dataframe
cohort_studies_sub = dict()

for cohort, dataset in zip(cohorts_sub, datasets_sub):
    cohort_studies_sub[cohort] = dataset.loc[dataset['Months']==0].copy() # reduce to BL visit

In [104]:
# add the preprocessed columns from sub table of each dataset to the main table of dataset
for i in cohort_studies:
#     cols = cohort_studies_sub[i].columns.difference(cohort_studies[i].columns)
    cols = ['Age', 'Sex', 'Education', 'APOE4', 'CDR', 'Race']
    
    for col in cols:
        
        if col in cohort_studies_sub[i].columns:
            cohort_studies[i][col] = cohort_studies_sub[i][col]

### Read harmonized mapping tables

In [105]:
modality = [pd.read_csv(file, sep=',') for file in sorted(glob.glob('../feature_tables' + "/*."+'csv'))]
name = [file.split(".")[0] for file in sorted(os.listdir('../feature_tables'))]

In [106]:
# make a dictionary that contains all modalities as a dataframe
mappings = dict()

for moda, na in zip(modality, name):
    mappings[na.split(' - ')[1]] = moda

In [107]:
harmonized_features = pd.concat(mappings, ignore_index=True) # combine all tables

In [108]:
# exclude categorical and taboo features
harmonized_features = harmonized_features.loc[(harmonized_features['Rank']!=1) & (harmonized_features['Rank']!=2)]

### Read the feature availability files for all cohorts

In [109]:
ava_mapp = [pd.read_csv(file, sep='\t') for file in sorted(glob.glob('../feature_availability_in_cohorts' + "/*."+'tsv'))]
tablesss = [file.split(".")[0] for file in sorted(os.listdir('../feature_availability_in_cohorts'))]

In [110]:
# make a dictionary that contains all modalities as a dataframe
available_features = dict()

for modal, df in zip(tablesss, ava_mapp):
    available_features[modal] = df

In [111]:
existing_features = pd.concat(available_features, ignore_index=True) # combine all tables

In [112]:
existing_features.replace({0: np.nan}, inplace=True) # 0 indicates that the feature was not measured 

### Read cutoffs obtained using all methods

In [113]:
table_method = [pd.read_csv(file, index_col=0, low_memory=False) for file in sorted(glob.glob('../results/cutoffs/' + "/*."+'csv'))]
method_name = [file.split(".")[0] for file in sorted(os.listdir('../results/cutoffs/'))]

In [114]:
# make a dictionary that contains each cutoff table as a dataframe
cutoffs_ = dict()

for tm, mn in zip(table_method, method_name):
    cutoffs_[mn] = tm

In [115]:
[cutoffs_['km_cutoffs'].rename(columns={col: col.split('_')[0]}, inplace=True) for col in cutoffs_['km_cutoffs'].columns]

[None, None, None]

### Selecetion of cohort studies for A/T/N assignment

### Select the patient that have CSF biomarker, disregard the diagnostic status

In [116]:
atn = pd.DataFrame(index=available_features['csf'].iloc[:3].replace({0: np.nan}).dropna(axis=1).columns[1:].to_list(), columns=mappings['csf'].Feature.loc[0:2].to_list()+(["Total"]))
# atn = pd.DataFrame(index=cohort_studies, columns=['A', 'T', 'N'])

In [117]:
for cohort in atn.index:
    for feat in mappings['csf'][cohort].loc[0:2].dropna().to_list():
        if feat in cohort_studies[cohort].columns:
            atn.loc[cohort, mappings['csf'].loc[mappings['csf'][cohort]==feat, 'Feature']] = len(cohort_studies[cohort][feat].dropna())
            atn.loc[cohort, 'Total'] = len(cohort_studies[cohort][mappings['csf'][cohort].loc[0:2].dropna().to_list()].dropna())

In [118]:
# atn

In [119]:
diag = pd.DataFrame(index=available_features['csf'].iloc[:3].replace({0: np.nan}).dropna(axis=1).columns[1:].to_list(), columns=cohort_studies['ADNI']['Diagnosis'].dropna().unique())

In [120]:
for cohort in diag.index:
    for dia in diag.columns:
        diag.loc[cohort, dia] = len(cohort_studies[cohort].loc[cohort_studies[cohort]['Diagnosis']==dia][mappings['csf'][cohort].loc[0:2].dropna().to_list()].dropna())

### Remove the empty columns from all cohorts that we are intrested in
### Remove the participant without all 3 CSF biomarkers

In [121]:
selected_cohorts = dict()

for coh in diag.index:
    selected_cohorts[coh] = cohort_studies[coh].dropna(axis=1, how='all')

In [122]:
total_feats = dict()

# existing_features.set_index('Feature', inplace=True)

for feat in existing_features.Feature:
    total_feats[feat] = existing_features.loc[existing_features.Feature==feat][selected_cohorts].dropna(axis=1).columns

In [123]:
for cohort in atn.index:
    feat = mappings['csf'][cohort].loc[0:2].dropna().to_list()
    cohort_studies[cohort] = cohort_studies[cohort].dropna(subset=feat)

As Some features have suffix due to merging tables for certain cohorts, first investigate if all the harmonized features are in cohorts. Rename the ones that have suffix so it can be compatible to work with our harmonized names.

In [124]:
cohort_studies['ADNI'] = cohort_studies['ADNI'].rename(columns={'PTEDUCAT_x': 'PTEDUCAT', 'TRABSCOR_bl': 'TRABSCOR', 'LDELTOTAL_BL': 'LDELTOTAL'})

### CSF biomarkers, two classes, normal vs abnormal

In [125]:
# modality_ = ['clinical_i', 'clinical_ii','hippocampus', 'csf']
# modality_ = ['basal_ganglia', 'brain_poles_volume', 'cerebellum', 'clinical_i', 
#             'clinical_ii', 'csf', 'diencephalus', 'general_brain', 
#             'mri_others', 'pet', 'plasma', 'ventricles', 'hippocampus']
modality_ = ['basal_ganglia', 'brain_poles_volume', 'cerebellum', 'csf', 'diencephalus', 'general_brain', 
            'mri_others', 'pet', 'plasma', 'ventricles', 'hippocampus']

selected_feat = dict()

for i in modality_:
    selected_feat[i] = mappings[i]

In [126]:
# make a dataframe containing all the mapped features
features_all = pd.concat(selected_feat, ignore_index=True)

In [127]:
features_all = features_all[atn.index.union(['Feature', 'Rank'])] # subset the cohorts of interest

In [128]:
# remove the features that are not available in all studies
features_all = features_all.loc[features_all['Feature'].isin(existing_features[atn.index.union(['Feature'])].dropna(how='all')['Feature'].to_list())]

### Convert MRI measurements to mm3, same as ADNI and other cohorts

In [129]:
mri_nacc = ['basal_ganglia', 'brain_poles_volume', 'cerebellum', 'diencephalus', 'general_brain', 
            'mri_others', 'pet', 'plasma', 'ventricles', 'hippocampus']

for i in mri_nacc:
    
    for variable in mappings[i][['Feature', 'NACC']].dropna()['Feature'].to_list():
        
        if "Volume" in variable:
            nacc_var = mappings[i].loc[mappings[i]['Feature']==variable, 'NACC']
            cohort_studies['NACC'][nacc_var] = cohort_studies['NACC'][nacc_var] * 1000

In [130]:
nacc_mri_to_convert = list()

for i in mri_nacc:
    
    for feat_ in mappings[i]['NACC'].dropna().to_list():
        nacc_mri_to_convert.append(feat_)

In [131]:
# cohort_studies['NACC'][nacc_mri_to_convert].dropna(how='all')

* Rank 1 --> Categorical features
* Rank 2 --> Taboo features: some categorical and some numerical
* Rank nan --> Numerical features 

replace nan with 0

In [132]:
features_all['Rank'].replace({np.nan: 0}, inplace=True)

In [133]:
features_all.replace({"No total score.": np.nan}, inplace=True)

In [134]:
# fix PharmaCog column names
for i in cohort_studies['PharmaCog'].columns:
    if "\xa0" in i:
        new = str(i).replace(u'\xa0', u'')
        cohort_studies['PharmaCog'].rename(columns={i: new}, inplace=True)

In [135]:
Counter(cohort_studies['PharmaCog']['Diagnosis'])

Counter({'MCI': 145})

## Prepare datasets

In [136]:
def select_atn_participants(dfss, cohorts, thresholds, features):
    """cohorts: list of cohort names 
       dfss: dictionary of cohorts where each key is the name of a cohort
       thresholds: cutoff values obtained using a methodology
       
       Select the features and participants and categorize the participant into ATN profiles using thresholds
       obtained from each methodology.
       
       return a df which contain the combination of paticipant from the selected cohorts while harmonizing
       the features names.
    """
    
    # make a list of additional features to be investigated 
#     additional_feat = ['Age', 'Sex', 'Education', 'APOE4', 'CDR', 'Race']
    additional_feat = ['APOE4']

    # make an empty dictionary to add the datasets to
    df_= pd.DataFrame(columns=set(features.loc[features['Rank']==0]
                                  [cohorts + ['Feature']].dropna()['Feature']).difference(['Feature'] + list(thresholds.columns)).union(additional_feat))
   
    for i in cohorts:
#         print(i)

        dfs = dict()
        dfs[i] = dfss[i].copy() # make a copy of the dataset of interest
        # select the subset of datasets with features to be investigated
        dfs[i] = dfs[i][features.loc[features['Rank']==0][cohorts + ['Feature']].dropna()[i].to_list() + additional_feat].dropna(axis=1, how='all')
        # rename all the columns so we can concat the datasets later
        [dfs[i].rename(columns={col: coln}, inplace=True) for col, coln in 
         zip(features.loc[features['Rank']==0][cohorts + ['Feature']].dropna()[i].to_list(), 
             features.loc[features['Rank']==0][cohorts + ['Feature']].dropna()['Feature'].to_list())]
        dfs[i]['Cohort'] = i # add a cohort name column
        # change the datatype to str as these are categorical features. astype doesn't work as it will include nan values
        dfs[i] = dfs[i].replace({'APOE4': {0.0: '0', 2.0: '2', 1.0: '1'}}) 

        if i!='NACC':
            
            for biomarker in thresholds.columns:
                # select the cutoff value for each biomarker for each cohort
                threshold = thresholds.loc[i][biomarker]

                # dichotomize the participants
                if biomarker == 'pTau in CSF': 
                    dfs[i].loc[dfs[i][biomarker]>threshold, "T"] = 'T+'
                    dfs[i].loc[dfs[i][biomarker]<threshold, "T"] = 'T-'

                elif biomarker == 'tTau in CSF': 
                    dfs[i].loc[dfs[i][biomarker]>threshold, "N"] = 'N+'
                    dfs[i].loc[dfs[i][biomarker]<threshold, "N"] = 'N-'

                else: 
                    dfs[i].loc[dfs[i][biomarker]<threshold, "A"] = 'A+'
                    dfs[i].loc[dfs[i][biomarker]>threshold, "A"] = 'A-'

            # join the 3 columns to make the final ATN categorie                                 
            dfs[i]['ATN'] = dfs[i]['A'] + dfs[i]['T'] + dfs[i]['N']
            # remove the columns that we are not interested in
            dfs[i] = dfs[i][dfs[i].columns.difference(['A', 'T', 'N', 'Mini-Mental State Examination (MMSE)'] + list(thresholds.columns))]
            
        else:
            
            elisa_index = cohort_studies[i].loc[(cohort_studies[i]['CSFTTMD']==1)].index
            xmap_index = cohort_studies[i].loc[(cohort_studies[i]['CSFTTMD']==2)].index
            
            for biomarker in thresholds.columns:
                # select the cutoff value for each biomarker for each cohort
                elisa = thresholds.loc[i + "_ELISA"][biomarker] # ELISA
                xmap = thresholds.loc[i + "_XMAP"][biomarker] #XMAP

                # dichotomize the participants
                if biomarker == 'pTau in CSF': 
                    
                    dfs[i].loc[(dfs[i].index.isin(elisa_index)) & (dfs[i][biomarker]>elisa), "T"] = 'T+'
                    dfs[i].loc[(dfs[i].index.isin(elisa_index)) & (dfs[i][biomarker]<elisa), "T"] = 'T-'
                    dfs[i].loc[(dfs[i].index.isin(xmap_index)) & (dfs[i][biomarker]>xmap), "T"] = 'T+'
                    dfs[i].loc[(dfs[i].index.isin(xmap_index)) & (dfs[i][biomarker]<xmap), "T"] = 'T-'

                elif biomarker == 'tTau in CSF': 
                    dfs[i].loc[(dfs[i].index.isin(elisa_index)) & (dfs[i][biomarker]>elisa), "N"] = 'N+'
                    dfs[i].loc[(dfs[i].index.isin(elisa_index)) & (dfs[i][biomarker]<elisa), "N"] = 'N-'
                    dfs[i].loc[(dfs[i].index.isin(xmap_index)) & (dfs[i][biomarker]>xmap), "N"] = 'N+'
                    dfs[i].loc[(dfs[i].index.isin(xmap_index)) & (dfs[i][biomarker]<xmap), "N"] = 'N-'

                else: 
                    dfs[i].loc[(dfs[i].index.isin(elisa_index)) & (dfs[i][biomarker]<elisa), "A"] = 'A+'
                    dfs[i].loc[(dfs[i].index.isin(elisa_index)) & (dfs[i][biomarker]>elisa), "A"] = 'A-'
                    dfs[i].loc[(dfs[i].index.isin(xmap_index)) & (dfs[i][biomarker]<xmap), "A"] = 'A+'
                    dfs[i].loc[(dfs[i].index.isin(xmap_index)) & (dfs[i][biomarker]>xmap), "A"] = 'A-'

            # join the 3 columns to make the final ATN categorie                                 
            dfs[i]['ATN'] = dfs[i]['A'] + dfs[i]['T'] + dfs[i]['N']
            # remove the columns that we are not interested in
            dfs[i] = dfs[i][dfs[i].columns.difference(['A', 'T', 'N', 'Mini-Mental State Examination (MMSE)', 'Trail Making Test (TMT) A', 'Verbal fluency tests (Semantic) Animal'] + list(thresholds.columns))]
            
#         print(dfs[i])

        
        df_ = pd.concat([df_, dfs[i]])
#         print(df_)
#         df_ = df_.dropna(axis=1, thresh=len(df_.index)/2)
#         print(df_)
    
    df_ = df_.dropna(axis=1, how='all')
    df_ = df_.dropna(axis=1, thresh=1200)

    df_ = df_.dropna()

    return df_

In [137]:
clustering_df_km = select_atn_participants(cohort_studies, ['ADNI', 'EDSD', 'ARWIBO', 'NACC', 'JADNI', 'DOD-ADNI', 'PharmaCog'], cutoffs_['km_cutoffs'], features_all)
clustering_df_gmm = select_atn_participants(cohort_studies, ['ADNI', 'EDSD', 'ARWIBO', 'NACC', 'JADNI', 'DOD-ADNI', 'PharmaCog'], cutoffs_['gmm_cutoffs'], features_all)

In [138]:
clustering_df_km

,Right Lateralorbitofrontal Mean Cortical Thickness,Right Parahippocampal Mean Cortical Thickness,Right Rostral Middle Frontal Mean Cortical Thickness,Left Paracentral Gray Matter Volume,Right Transverse Temporal Grey Matter Volume,Left Pars Opercularis Mean Cortical Thickness,Left Posterior Cingulate Mean Cortical Thickness,Left Lateraloccipital Mean Cortical Thickness,Left Transverse Temporal Grey Matter Volume,Left Transverse Temporal Mean Cortical Thickness,...,Left Cuneus Mean Cortical Thickness,Right Superiorfrontal Mean Cortical Thickness,Left Superior Parietal Gray Matter Volume,Right Isthmus Cingulate Mean Cortical Thickness,Left Pericalcarine Mean Cortical Thickness,Right Pericalcarine Mean Cortical Thickness,Right Superior Temporal Mean Cortical Thickness,Right Lateraloccipital Mean Cortical Thickness,ATN,Cohort
2002,2.551,2.495,2.121,2940.0,1039.0,2.516,2.713,2.379,1312.0,2.221,...,1.933,2.576,12473.0,2.459,1.595,1.482,2.769,2.452,A-T-N-,ADNI
2022,2.463,3.077,2.205,2784.0,768.0,2.446,2.488,2.223,1062.0,2.131,...,1.644,2.566,13904.0,2.438,1.360,1.411,2.648,2.157,A+T-N-,ADNI
2026,2.445,2.507,1.971,2355.0,571.0,2.219,2.129,2.008,827.0,1.992,...,1.734,2.168,10199.0,2.065,1.253,1.332,2.437,1.984,A-T-N-,ADNI
2031,2.377,3.016,2.082,2499.0,778.0,2.399,2.922,1.955,879.0,1.928,...,1.529,2.388,12750.0,2.471,1.335,1.313,2.669,2.039,A-T-N-,ADNI
2042,2.428,2.637,2.388,3372.0,621.0,2.585,2.230,1.973,1399.0,2.282,...,1.813,2.612,12343.0,2.022,1.543,1.660,2.671,2.031,A+T-N-,ADNI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PT-37-024,2.470,2.160,2.280,1469.0,506.0,1.680,1.890,1.900,551.0,1.540,...,1.480,2.440,6353.0,1.810,1.220,1.310,2.340,1.800,A+T-N-,PharmaCog
PT-37-025,2.170,2.020,1.860,2476.0,611.0,2.140,2.010,1.920,914.0,2.010,...,1.540,2.090,9683.0,1.770,1.360,1.210,2.220,1.990,A-T-N-,PharmaCog
PT-37-026,2.540,2.250,2.250,3120.0,814.0,2.360,2.540,1.990,1037.0,2.510,...,1.580,2.540,11016.0,2.300,1.470,1.580,2.790,2.090,A+T+N+,PharmaCog
PT-37-027,2.540,2.580,2.180,3467.0,797.0,2.300,2.430,2.060,794.0,1.600,...,1.740,2.390,11785.0,2.550,1.480,1.500,2.710,2.130,A+T-N-,PharmaCog


#### exclude CSF Volume as it seems wrong in the NACC dataset and that could potentially add bias to our analysis

In [139]:
clustering_df_km = clustering_df_km[clustering_df_km.columns.difference(['Cerebrospinal Fluid Volume'])]
clustering_df_gmm = clustering_df_gmm[clustering_df_gmm.columns.difference(['Cerebrospinal Fluid Volume'])]

In [140]:
def purity_score(y_true, y_pred):
    # compute contingency matrix (also called confusion matrix)
    contingency_matrix = metrics.cluster.contingency_matrix(y_true, y_pred)
    # return purity
    return np.sum(np.amax(contingency_matrix, axis=0)) / np.sum(contingency_matrix) 

In [141]:
def atn_based_df(df):
    """"""
    
    labels_atn = dict()
    score_atn = dict()
    
    # change the cohort names to int and write in a column
    for i, j in zip(df.Cohort.unique(), range(len(df.Cohort.unique()))):
        df.loc[df['Cohort']==i, 'Cohort_number'] = j
    
    # select the profiles that have over 20 participants
    # make a dictionary where the selcted ATN profiles are key and 
    # the subset of dataframe categorized in that ATN profile is value
    dfs_sub = {atn: df.loc[df.ATN==atn].copy() for atn in list((a) for a,b in dict(Counter(df.ATN)).items() if b >20)}
    
    for i in dfs_sub:
#         print(i, len(dfs_sub[i].Cohort.unique()))
        hierarchical_cluster = AgglomerativeClustering(n_clusters=len(dfs_sub[i].Cohort.unique()), affinity='euclidean', linkage='ward')
        labels = hierarchical_cluster.fit_predict(dfs_sub[i][dfs_sub[i].columns.difference(['ATN', 'Cohort', 'Cohort_number'])])
        labels_atn[i] = labels
        score_atn[i] = purity_score(dfs_sub[i]['Cohort_number'], labels)
     
    return labels_atn, score_atn, dfs_sub

In [142]:
labels_km, score_km, atn_km_df = atn_based_df(clustering_df_km)
labels_gmm, score_gmm, atn_gmm_df = atn_based_df(clustering_df_gmm)

In [143]:
# #test
# clustering_df_km_t = clustering_df_km[clustering_df_km['Cohort']!='NACC'].copy()
# clustering_df_gmm_t = clustering_df_gmm[clustering_df_gmm['Cohort']!='NACC'].copy()

In [144]:
# #test
# labels_km_t, score_km_t, atn_km_df_t = atn_based_df(clustering_df_km_t)
# labels_gmm_t, score_gmm_t, atn_gmm_df_t = atn_based_df(clustering_df_gmm_t)

In [145]:
# print("Cluster purity (K-means):")
# score_km

In [146]:
# print("Cluster purity (GMM):")
# score_gmm

In [147]:
for i in labels_km:
    atn_km_df[i].loc[:, 'predicted_cohort'] = list(labels_km[i])

for i_ in labels_gmm:
    atn_gmm_df[i_].loc[:, 'predicted_cohort'] = list(labels_gmm[i_])

In [148]:
# #test
# for i in labels_km_t:
#     atn_km_df_t[i].loc[:, 'predicted_cohort'] = list(labels_km_t[i])

# for i_ in labels_gmm_t:
#     atn_gmm_df_t[i_].loc[:, 'predicted_cohort'] = list(labels_gmm_t[i_])

In [149]:
def calculate_cramer(dfs):
    """ """
    results = dict()
    
    for i in dfs:
        df_ = dfs[i].set_index('Cohort')
        mat = pd.crosstab(df_.index, df_['predicted_cohort'])
#         print(i, mat)
#         print(i, round(stats.contingency.association(mat, method='cramer'), 2))
        results[i] = round(stats.contingency.association(mat, method='cramer'), 2)
        
    return results

In [150]:
pd.DataFrame.from_dict(calculate_cramer(atn_km_df), orient='index').transpose()[['A-T-N-', 'A-T+N+', 'A-T-N+', 'A+T+N-', 'A+T-N-', 'A+T-N+', 'A+T+N+']]

,A-T-N-,A-T+N+,A-T-N+,A+T+N-,A+T-N-,A+T-N+,A+T+N+
0,0.2,0.31,0.52,0.32,0.3,0.38,0.28


In [151]:
pd.DataFrame.from_dict(calculate_cramer(atn_gmm_df), orient='index').transpose()[['A-T-N-', 'A-T+N+', 'A-T-N+', 'A+T+N-', 'A+T-N-', 'A+T-N+', 'A+T+N+']]

,A-T-N-,A-T+N+,A-T-N+,A+T+N-,A+T-N-,A+T-N+,A+T+N+
0,0.21,0.59,0.58,0.62,0.28,0.33,0.27


In [152]:
def calculate_chi2(dfs):
    """ """
    results_chi2 = dict()
    results_p = dict()
    results_df = dict()
    results_ex = dict()
    
    for i in dfs:
        df_ = dfs[i].set_index('Cohort')
        mat = pd.crosstab(df_.index, df_['predicted_cohort'])
        results_chi2[i], results_p[i], results_df[i], results_ex[i] = stats.chi2_contingency(mat, correction=True)
        
    return results_chi2, results_p, results_df, results_ex

In [153]:
results_chi2_km, results_p_km, results_df_km, results_ex_km = calculate_chi2(atn_km_df)
results_chi2_gmm, results_p_gmm, results_df_gmm, results_ex_gmm = calculate_chi2(atn_gmm_df)

In [154]:
print(results_chi2_km)
print("    ")
print(results_p_km)
print("    ")
print(results_df_km)

{'A-T-N-': 89.25610204990821, 'A+T-N-': 218.09962970174868, 'A+T+N+': 167.92391125097416, 'A-T+N+': 28.692343717790145, 'A+T+N-': 35.505407169990505, 'A-T-N+': 29.40123456790123, 'A+T-N+': 28.685219542362397}
    
{'A-T-N-': 2.042496476747995e-06, 'A+T-N-': 6.332504768791712e-28, 'A+T+N+': 6.1771699392248855e-19, 'A-T+N+': 0.276980979771904, 'A+T+N-': 0.49194109345408876, 'A-T-N+': 0.24749241674394443, 'A+T-N+': 0.2772883312718669}
    
{'A-T-N-': 36, 'A+T-N-': 36, 'A+T+N+': 36, 'A-T+N+': 25, 'A+T+N-': 36, 'A-T-N+': 25, 'A+T-N+': 25}


In [162]:
results_p_gmm

{'A-T-N-': 2.5015117558273577e-09,
 'A+T-N-': 3.5193231396238735e-33,
 'A+T+N+': 2.867913391022302e-08,
 'A-T+N+': 0.0018565188134793346,
 'A+T+N-': 1.817626616072492e-05,
 'A+T-N+': 0.0802408877031763,
 'A-T-N+': 0.056990382207007244}

In [163]:
for i in ['A-T-N-', 'A-T+N+', 'A-T-N+', 'A+T+N-', 'A+T-N-', 'A+T-N+', 'A+T+N+']:
    print(i, round(results_p_gmm[i], 3))

A-T-N- 0.0
A-T+N+ 0.002
A-T-N+ 0.057
A+T+N- 0.0
A+T-N- 0.0
A+T-N+ 0.08
A+T+N+ 0.0


In [155]:
print(results_chi2_gmm)
print("    ")
print(results_p_gmm)
print("    ")
print(results_df_gmm)

{'A-T-N-': 109.38306013989786, 'A+T-N-': 246.40864218054207, 'A+T+N+': 102.28110579852627, 'A-T+N+': 37.37500000000001, 'A+T+N-': 50.6313957535677, 'A+T-N+': 35.4576931583628, 'A-T-N+': 37.05952380952382}
    
{'A-T-N-': 2.5015117558273577e-09, 'A+T-N-': 3.5193231396238735e-33, 'A+T+N+': 2.867913391022302e-08, 'A-T+N+': 0.0018565188134793346, 'A+T+N-': 1.817626616072492e-05, 'A+T-N+': 0.0802408877031763, 'A-T-N+': 0.056990382207007244}
    
{'A-T-N-': 36, 'A+T-N-': 36, 'A+T+N+': 36, 'A-T+N+': 16, 'A+T+N-': 16, 'A+T-N+': 25, 'A-T-N+': 25}


### Note: One could use the crosstab instead of the function below

In [50]:
def count_clusters(df):
    
    """df: dataframe with ATN categorization using certain method and containing cluster labels
       return: the number of participant within each cohort and within each ATN profile assigned to each labels"""
    
    # make an empty dictionary of dataframes to store the results
    clustering_result_ = {i: pd.DataFrame(index=df[i].Cohort.unique(), 
                                            columns=sorted(df[i].predicted_cohort.unique())) for i in df}
    
    # check the number of participants clustered to each coohort within each biomarker profile
    for profi in df:
    
        for name in df[profi]['Cohort'].unique():
            clustering_result_[profi].loc[name] = Counter(df[profi].loc[df[profi]['Cohort']==name, 'predicted_cohort'])
            
            
    # replace all nan enteries with 0
    [clustering_result_[i].replace({np.nan: 0}, inplace=True) for i in clustering_result_]
    
    # change enteries to integer
    for i in clustering_result_:
        clustering_result_[i] = clustering_result_[i].astype(int)

    return clustering_result_

In [51]:
clustering_result_km = count_clusters(atn_km_df)
clustering_result_gmm = count_clusters(atn_gmm_df)

In [52]:
print("K-Means")
for i in clustering_result_km.keys():
    print(len(clustering_result_km[i].index), list(clustering_result_km[i].index))
    
print("GMM")
for i in clustering_result_gmm.keys():
    print(len(clustering_result_gmm[i].index), list(clustering_result_gmm[i].index))

K-Means
7 ['ADNI', 'EDSD', 'ARWIBO', 'NACC', 'JADNI', 'DOD-ADNI', 'PharmaCog']
7 ['ADNI', 'EDSD', 'ARWIBO', 'NACC', 'JADNI', 'DOD-ADNI', 'PharmaCog']
7 ['ADNI', 'EDSD', 'ARWIBO', 'NACC', 'JADNI', 'DOD-ADNI', 'PharmaCog']
6 ['ADNI', 'EDSD', 'ARWIBO', 'NACC', 'DOD-ADNI', 'PharmaCog']
7 ['ADNI', 'EDSD', 'ARWIBO', 'NACC', 'JADNI', 'DOD-ADNI', 'PharmaCog']
6 ['ADNI', 'ARWIBO', 'NACC', 'JADNI', 'DOD-ADNI', 'PharmaCog']
6 ['ADNI', 'EDSD', 'ARWIBO', 'NACC', 'JADNI', 'PharmaCog']
GMM
7 ['ADNI', 'EDSD', 'ARWIBO', 'NACC', 'JADNI', 'DOD-ADNI', 'PharmaCog']
7 ['ADNI', 'EDSD', 'ARWIBO', 'NACC', 'JADNI', 'DOD-ADNI', 'PharmaCog']
7 ['ADNI', 'EDSD', 'ARWIBO', 'NACC', 'JADNI', 'DOD-ADNI', 'PharmaCog']
5 ['ADNI', 'ARWIBO', 'NACC', 'DOD-ADNI', 'PharmaCog']
5 ['ADNI', 'EDSD', 'NACC', 'JADNI', 'PharmaCog']
6 ['ADNI', 'EDSD', 'ARWIBO', 'NACC', 'JADNI', 'PharmaCog']
6 ['ADNI', 'EDSD', 'ARWIBO', 'NACC', 'JADNI', 'PharmaCog']


In [53]:
clustering_result_km.keys()

dict_keys(['A-T-N-', 'A+T-N-', 'A+T+N+', 'A-T+N+', 'A+T+N-', 'A-T-N+', 'A+T-N+'])

In [54]:
clustering_result_gmm.keys()

dict_keys(['A-T-N-', 'A+T-N-', 'A+T+N+', 'A-T+N+', 'A+T+N-', 'A+T-N+', 'A-T-N+'])

In [55]:
clustering_result_gmm['A+T-N+']

,0,1,2,3,4,5
ADNI,1,2,1,2,2,0
EDSD,0,2,0,3,2,1
ARWIBO,7,1,1,1,4,0
NACC,7,3,7,7,3,0
JADNI,1,3,0,0,1,0
PharmaCog,0,2,2,1,0,0
